##### Copyright 2018 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# tf.data を使ったテキストの読み込み

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

このチュートリアルでは、`tf.data.TextLineDataset` を使ってテキストファイルからサンプルを読み込む方法を例示します。`TextLineDataset` は、テキストファイルからデータセットを作成するために設計されています。この中では、元のテキストファイルの一行一行がサンプルです。これは、（たとえば、詩やエラーログのような）基本的に行ベースのテキストデータを扱うのに便利でしょう。

このチュートリアルでは、おなじ作品であるホーマーのイリアッドの異なる 3 つの英語翻訳版を使い、テキスト 1 行から翻訳者を特定するモデルを訓練します。

## 設定

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-rc1

import tensorflow as tf

import tensorflow_datasets as tfds
import os

3 つの翻訳のテキストは次のとおりです。

 - [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

 - [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

このチュートリアルで使われているテキストファイルは、ヘッダ、フッタ、行番号、章のタイトルの削除など、いくつかの典型的な前処理を行ったものです。前処理後のファイルをダウンロードしましょう。

In [0]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

## テキストをデータセットに読み込む

ファイルをイテレートし、それぞれを別々のデータセットに読み込みます。

サンプルはそれぞれにラベル付けが必要なので、ラベル付け関数を適用するために `tf.data.Dataset.map` を使います。このメソッドは、データセット中のすべてのサンプルをイテレートし、(`example, label`)というペアを返します。

In [0]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

ラベル付けの終わったデータセットを結合して一つのデータセットにし、シャッフルします。

In [0]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [0]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

`tf.data.Dataset.take` と `print` を使って、`(example, label)` のペアがどのようなものかを見ることができます。`numpy` プロパティがそれぞれのテンソルの値を示します。

In [0]:
for ex in all_labeled_data.take(5):
  print(ex)

## テキスト行を数字にエンコードする

機械学習モデルが扱うのは単語ではなくて数字であるため、文字列は数字のリストに変換する必要があります。このため、一意の単語を一意の数字にマッピングします。

### ボキャブラリーの構築

まず最初に、テキストをトークン化し、個々の一意な単語の集まりとして、ボキャブラリーを構築します。これを行うには、TensorFlow やPython を使ういくつかの方法があります。ここでは次のようにします。

1. 各サンプルの `numpy` 値をイテレートします。
2. `tfds.features.text.Tokenizer` を使って、それをトークンに分割します。
3. 重複を排除するため、トークンを Python の集合に集約します。
4. あとで使用するため、ボキャブラリーのサイズを取得します。

In [0]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

### サンプルをエンコードする

`vocabulary_set` を `tfds.features.text.TokenTextEncoder` に渡してエンコーダーを作成します。エンコーダーの `encode` メソッドは、テキスト文字列を引数にとり、整数のリストを返します。

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

1行だけにこれを適用し、出力がどの様になるか確かめることができます。

In [0]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

In [0]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

次に、このエンコーダーを `tf.py_function` でラッピングして、データセットの `map` メソッドに渡し、データセットに適用します。

In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = all_labeled_data.map(encode_map_fn)

## データセットを、テスト用と訓練用のバッチに分割する

`tf.data.Dataset.take`と`tf.data.Dataset.skip`を使って、小さなテスト用データセットと、より大きな訓練用セットを作成します。

モデルに渡す前に、データセットをバッチ化する必要があります。通常、バッチの中のサンプルはおなじサイズと形状である必要があります。しかし、これらのデータセットの中のサンプルはすべておなじサイズではありません。テキストの各行の単語数は異なっています。このため、（`batch`の代わりに）`tf.data.Dataset.padded_batch` メソッドを使ってサンプルをおなじサイズにパディングします。

In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

もう、`test_data` と `train_data` は、(`example, label`)というペアのコレクションではなく、バッチのコレクションです。それぞれのバッチは、（**たくさんのサンプル**, **たくさんのラベル**）という配列のペアです。

見てみましょう。

In [0]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

（ゼロをパディングに使用した）新しいトークン番号を1つ導入したので、ボキャブラリーサイズは1つ増えています。

In [0]:
vocab_size += 1

## モデルを構築する

In [0]:
model = tf.keras.Sequential()

最初の層は、整数表現を密なベクトル埋め込みに変換します。詳細は[単語埋め込み](../../tutorials/sequences/word_embeddings)のチュートリアルを参照ください。

In [0]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

次の層は[Long Short-Term Memory](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) 層です。この層により、モデルは単語をほかの単語の文脈の中で解釈します。LSTM の Bidirectional ラッパーにより、データポイントを、その前とその後のデータポイントとの関連で学習することができます。

In [0]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

最後に、一つ以上の全結合層があり、最後の層は出力層です。出力層はラベルすべての確率を生成します。もっと複雑なも確率の高いラベルが、モデルが予測するサンプルのラベルです。

In [0]:
# 1 つ以上の Dense 層
# `for` 行の中のリストを編集して、層のサイズの実験をしてください
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# 出力層 最初の引数はラベルの数
model.add(tf.keras.layers.Dense(3, activation='softmax'))

最後にモデルをコンパイルします。ソフトマックスによるカテゴリー分類モデルでは、損失関数として `sparse_categorical_crossentropy` を使用します。ほかのオプティマイザを使うこともできますが、`adam` がよく使われます。

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## モデルを訓練する

このモデルをこのデータに適用すると（約83%の）まともな結果が得られます。

In [0]:
model.fit(train_data, epochs=3, validation_data=test_data)

In [0]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))